Импортируем библиотеки

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from catboost import CatBoostClassifier

Считываем данные

In [2]:
X = pd.read_parquet("../../data/input_data/train_ai_comp_final_dp.parquet")

y = X["target"]
X.drop(columns = ["sample_ml_new", "target", "id"], inplace = True)

In [10]:
print(X.shape)
X.tail(3)

(519615, 1076)


,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
693039,1761,1759,0,168,191,1,125,290,298,176,...,0,0,0,0,0,51714,0,0,0,0
693040,1761,1759,141,74,191,1,125,290,298,176,...,0,0,0,0,0,51714,0,0,0,0
693042,1761,1759,141,28,191,0,125,290,298,176,...,0,0,0,0,50250,51714,0,0,0,0


In [9]:
y.tail(3)

693039    0
693040    0
693042    1
Name: target, dtype: int64

Разделяем данные на обучающую и тестовую выборки

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=160_032_240)

Обучаем модель с помощью градиентного бустинга

Вопросы на подумать: как учитывать категориальные фичи, как не переобучать модель и при чем тут валидационные данные, как отобрать фичи?

In [12]:
model = CatBoostClassifier()
model.fit(X_train, y_train, verbose=False)

Смотрим на результаты обучения

In [13]:
pred = model.predict_proba(X_test)

pred = pred[:, 1]
pred_binary = (pred >= 0.1)

In [14]:
print("F1_SCORE", f1_score(y_test, pred_binary))
print("PRECISION", precision_score(y_test, pred_binary))
print("RECALL", recall_score(y_test, pred_binary))
print("ROC_AUC", roc_auc_score(y_test, pred))

F1_SCORE 0.19214790195263817
PRECISION 0.15622361087654113
RECALL 0.24952792015106556
ROC_AUC 0.7497571983119106


Теперь сделаем сабмит

Для этого берем тестовые данные

In [15]:
X = pd.read_parquet("test_sber.parquet")
X.drop(columns = ["sample_ml_new", "id"], inplace = True)

In [19]:
print(X.shape)
X.tail(3)

(173433, 1076)


,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
173430,1761,1759,141,74,191,1,125,290,298,176,...,0,0,0,0,0,51714,0,0,0,0
173431,1761,1759,141,107,191,1,125,290,298,176,...,0,0,0,0,0,51714,0,0,0,0
173432,1761,1759,141,23,191,0,125,290,298,176,...,0,0,0,0,50250,51714,0,0,0,0


Предсказываем ответы для тестовой выборки

In [22]:
pred = model.predict_proba(X)

pred = pred[:, 1]
pred_binary = (pred >= 0.1).astype(int)

In [24]:
submission = pd.read_csv("../../data/input_data/sample_submission.csv")
submission.head(3)

,id,target_bin,target_prob
0,3,0,0.03
1,4,0,0.03
2,12,1,0.03


In [27]:
submission["target_prob"] = pred
submission["target_bin"] = pred_binary
submission.head(3)

,id,target_bin,target_prob
0,3,0,0.011261
1,4,0,0.017531
2,12,0,0.030847


In [37]:
submission.to_csv("../../data/output_data/baseline_submission.csv", index=False)

---